# TP SQL - Postgres/Postgis - M1 GLET

La réalisation de ce TP suppose que vous ayez déjà lu et mis en oeuvre le document word d'instruction pour l'installation de l'environnement de travail.

Le TP est à rendre avant le vendredi 15 mai 17h00 par un envoi du notebook sur ma boite mail.

En cas de besoin envoyez moi un mail à n.dhuygelaere@oieau.fr ou contactez moi via skype à n.dhuygelaere.

## Comment fonctionne un Notebook ?

Bonne question!

Je vous invite à visionner cette vidéo : https://www.youtube.com/watch?v=qZiN0dzx8ag qui vous expliquera les différentes fonctions de base et notamment comment créer et executer des cellules. Le but de ce TP n'est pas de faire de vous des pro de la programmation python mais de vous faire travailler le SQL avec postgres. Ainsi, vous écrivez les requêtes et je vous montre différentes utilisations possibles de vos données avec les librairies pythons.

Bon courage !

**IL EST INDISPENSABLE D'EXECUTER TOUTES LES CELLULES AU FUR ET A MESURE !**

Il est possible d'éxecuter toutes les cellules en utilisant l'entrée de menu **Kernel > Restart & Run All**

## Chargement des libraires python

In [1]:
#Cette première cellule permet de charger en mémoire toutes les librairies qui vous seront utiles
%matplotlib inline
from matplotlib import pyplot as plt
import psycopg2 as pg
import numpy as np
import pandas.io.sql as psql
import pandas as pd
import folium
from IPython.display import display
from IPython.display import HTML

## Connexion à la base

Si vous avez suivi les instructions d'installation contenues dans le document word, vous devez avoir un serveur postgres actif sur votre machine (en localhost) qui fonctionne sur le port **5432** et dont le compte du super admin est **postgres/postgres**

In [2]:
#Paramètres de connexion à la base postgres
conn = pg.connect("dbname=tp_sql port=5432 user=postgres password=postgres")

#La requête suivante va nous permettre de tester le bon fonctionnement de la connexion 
# en affichant le code et le nom des 10 premiers tuples de la table sous_secteur
# La clause SQL LIMIT permet de ne retourner que les 10 premiers tuples
test = psql.read_sql("SELECT cdsoussect,lbsoussect  FROM sous_secteur LIMIT 10", conn)
test


,cdsoussect,lbsoussect
0,K75,l 'indre de l 'indrois (nc) au vieux cher (c)
1,V60,L'Ouvèze de sa source à la Nesque
2,K36,l 'allier de la bieudre (c) à la loire (nc)
3,E63,Canal de la Somme du confluent du Canal du Nor...
4,L51,la gartempe de l 'ardour (nc) au vincou (nc)
5,B12,le Vair
6,H72,La Seine du confluent du Canal de Retour d'Eau...
7,M06,la sarthe de la vegre (nc) à la voutonne (c)
8,A16,l'Ill du confluent de la Lauch au confluent de...
9,V21,L'Anguillon


## Quelques astuces python

* Pour afficher le contenu d'une variable il suffit simplement d'écrire son nom sur la dernière ligne de la cellule (comme dans le cas ci-dessus ou la dernière ligne est "test") ou d'utiliser la fonction print

In [ ]:
print("Salut")
test

* Pour afficher uniquement les premières lignes d'un dataframe (un tabeau), il faut utiliser la fonction **head()** avec le nombre de lignes souhaitées à notre tableau: ex : test.head(3)

In [ ]:
test.head(3)


* Pour afficher uniquement les dernières lignes d'un dataframe (un tabeau), il faut utiliser la fonction **tail()** avec le nombre de lignes souhaitées à notre tableau: ex : test.tail(3)

In [ ]:
test.tail(3)

* Pour connaitre la liste des colonnes d'un dataframe (un tabeau), il faut utiliser la propriété **column**

In [ ]:
test.columns

* Pour connaitre les types des colonnes d'un dataframe (un tabeau), il faut utiliser la propriété **dtypes**

In [ ]:
test.dtypes

## Et maintenant jouons !

1. Afficher la liste des colonnes des tables **"sous_secteur", "obstacles" et "coursdeau"** à l'aide d'une requête de type "SELECT * FROM *ma_table*"

In [ ]:
#1.1 sous_secteur

In [ ]:
#1.2 obstacles

In [ ]:
#1.3 coursdeau

2. Compter le nombre d'élément contenu dans chacune des tables,

In [ ]:
#2.1 sous_secteur

In [ ]:
#2.2 obstacles

In [ ]:
#2.3 coursdeau

3. Compter le nombre d'obstacles à l'écoulement par département (grouper par la colonne **"cddepartem"** et utiliser l'alias **"nb_obs"** pour le résultat du comptage) et stocker le résultat dans la variable **"nb_obs_dpt"**

In [ ]:
#3
nb_obs_dpt = psql.read_sql("Votre requete ici", conn)

A l'aide du résultat du 3ème point, nous allons afficher un histogramme de notre résultat.
**Note** : Pour ceux qui veulent aller plus loin dans la mise en forme du graphique, la doc est consultable ici : https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.bar.html

In [ ]:
plt.rcParams["figure.figsize"] = (18,7)
nb_obs_dpt.plot.bar(width=1, color='orange')
plt.show()


4. Selectionner tous les obstacles à l'écoulement du Loiret (département 45) et conserver les résulats dans la variable **obs_45**

In [ ]:
obs_45 = psql.read_sql("Votre requete ICI", conn)
obs_45

5. Nous allons utiliser le résultat du point 4 pour faire une carte dynamique avec folium. Pour celà il faut convertir les coordonnées géographiques du champ geom en WGS 84 et dans un format utilisable par folium c'est à dire des coordonnées X et Y.

Postgres conserve ses données géographiques au format "geometry" ou "geography". Il faut donc convertir le champ "geom" en une colonne X et en une colonne Y. Par ailleurs, nos données sont en lambert 93 alors que folium attend ses coordonnées en WGS 84. 
* st_transform() : https://postgis.net/docs/ST_Transform.html
* st_x() : https://postgis.net/docs/ST_X.html
* st_y() : https://postgis.net/docs/ST_Y.html

In [5]:
obs_45 = psql.read_sql("SELECT cdobstecou, st_x(st_transform(geom, 4326)) as x, st_y(st_transform(geom, 4326)) as y FROM obstacles WHERE cddepartem = '45'", conn)
obs_45

,cdobstecou,x,y
0,ROE37282,1.766796,47.676782
1,ROE46534,2.392463,48.201041
2,ROE46918,2.763747,48.099990
3,ROE46919,2.752201,48.074314
4,ROE44230,2.280936,47.851250
...,...,...,...
1311,ROE118765,2.908345,47.932515
1312,ROE118766,2.994146,47.941480
1313,ROE118767,3.010695,47.940322
1314,ROE118768,2.831386,47.946992


In [6]:
display(HTML("<h2>Carte des obstacles à l'écoulement du Loiret</h2>"))
lat_mean = np.mean(obs_45.y)
lon_mean = np.mean(obs_45.x)
print("Centre de la carte (lat, lon)", lat_mean, lon_mean)

map = folium.Map(
    tiles='OpenStreetMap',
    location=(lat_mean, lon_mean),
    zoom_start=9
)
folium.LayerControl().add_to(map)
map_tooltip = 'Click me!'
#Add markers on map
for lat, lon, code in zip(obs_45.y, obs_45.x, obs_45.cdobstecou):
    folium.Marker(
        [lat, lon], 
        popup=f'<b><a target="_blank" href="http://www.sandre.eaufrance.fr/urn.php?urn=urn:sandre:donnees:ObstEcoul:FRA:code:{code}:::::html">{code}</a></b>',
        tooltip=map_tooltip
    ).add_to(map)

display(map)

Centre de la carte (lat, lon) 47.87841744502571 2.4572159291882554
